# BEZIEHEN VON REZENZIONEN GOOGLE MAPS

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np
import os, glob
import os.path

In [2]:
kliniklist = pd.read_excel(r'..\Klinikliste.xlsx')
df_kliniklist = pd.DataFrame(kliniklist, columns= ['Klinikname','Link Google Maps','Link Klinikbewertungen'])
df_kliniklist['klinik_ID'] = [i for i in range(1,25)]
df_kliniklist.reset_index(drop=True, inplace=True)

In [3]:
df_kliniklist.head(30)

,Klinikname,Link Google Maps,Link Klinikbewertungen,klinik_ID
0,Augenklinik Dr.Hoffmann,https://www.google.com/maps/place/Augenklinik+...,https://www.klinikbewertungen.de/klinik-forum/...,1
1,Krankenhaus Marienstift Braunschweig,https://www.google.com/maps/place/Krankenhaus+...,https://www.klinikbewertungen.de/klinik-forum/...,2
2,Herzogin Elisabeth Hospital Braunschweig,https://www.google.com/maps/place/Herzogin+Eli...,https://www.klinikbewertungen.de/klinik-forum/...,3
3,Asklepios Fachklinikum Göttingen,https://www.google.com/maps/place/Asklepios+Fa...,https://www.klinikbewertungen.de/klinik-forum/...,4
4,Asklepios Fachklinikum Tiefenbrunn,https://www.google.com/maps/place/Asklepios+Fa...,https://www.klinikbewertungen.de/klinik-forum/...,5
5,Friederikenstift Hannover,https://www.google.com/maps/place/DIAKOVERE+Fr...,https://www.klinikbewertungen.de/klinik-forum/...,6
6,Annastift Hannover,https://www.google.com/maps/place/DIAKOVERE+An...,https://www.klinikbewertungen.de/klinik-forum/...,7
7,Clementinenhaus,https://www.google.com/maps/place/DRK-Krankenh...,https://www.klinikbewertungen.de/klinik-forum/...,8
8,Sophienklinik Hannover,https://www.google.com/maps/place/Sophienklini...,https://www.klinikbewertungen.de/klinik-forum/...,9
9,KRH Klinikum Großburgwedel,https://www.google.com/maps/place/KRH+Klinikum...,https://www.klinikbewertungen.de/klinik-forum/...,10


#### Open the Comments of a KH

#### Accept the Cookies

In [4]:
def accept_cookies():
    try:
        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//iframe")))
        agree = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="introAgreeButton"]/span/span')))
        agree.click()
    except:
        driver.quit()

    driver.maximize_window()

#### Scrolling Down til Ends

In [5]:


def scroll_down(bew_anzahl):
    x=0
    while (x<(bew_anzahl//10+2)):
        scrollable_div = driver.find_element_by_css_selector('div.section-layout.section-scrollbox.scrollable-y.scrollable-show') # It gets the section of the scroll bar.
        time.sleep(2)
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)# Scroll it to the bottom.
        x=x+1
    
#scroll_down(bew_anzahl)
#bew_anzahl_2 = scroll_down(bew_anzahl)

#### Click to See All Comments

In [6]:
def see_all_comments():
    try:
        mehr=driver.find_elements_by_class_name('blue-link')
        for i in mehr:
            i.click()
    except:
        pass

#### Saving All Comments in a List

In [7]:
def saving_all_comments_in_a_list():
    r = BeautifulSoup(driver.page_source, 'html.parser')
    rlist = r.find_all('div', class_='section-review-content')

    id_r = r.find('button', 
                  class_='section-review-action-menu')['data-review-id']

    username = r.find('div', 
                      class_='section-review-title').find('span').text
    try:
        bewertung = r.find_all('span', class_='section-review-text')
    except Exception:
        review_text = None

    sterne = r.find_all('span', class_='section-review-stars')
    bewertung_datum = r.find_all('div', class_='section-review-metadata section-review-metadata-with-note')
    #likes=r.find_all('span', class_ ='section-review-thumbs-up-count')
    like=driver.find_elements_by_css_selector(".section-review-interactions-label>span:nth-child(2)")
    rel_date = r.find('span', class_='section-review-publish-date').text
    return bewertung, sterne, bewertung_datum, like

In [8]:
def exchange_column(old_df, new_df, column):
    old_df.drop(labels=column, axis="columns", inplace=True)
    old_df[column] = new_df[column]
    new_df = pd.DataFrame()

#### Collect All Likes

In [9]:
def collect_all_likes(like):
    count = 0
    likes = []
    for i in like:
        if i.text is str(''):
            #print('null')
            likes.append(None)
        else:
            likes.append(i.text)
            #print (i.text)
        count +=1   
    while(count<bew_anzahl):
        count+=1
        likes.append(None)
        #print('null')

    temp_df['Likes'] =  likes
    exchange_column(df,temp_df,'Likes')

#### Collect All Bewerbung_Datum

In [10]:
def collect_all_bewerbung_datum(bewertung_datum):
    bew_datum = []
    for i in bewertung_datum:
        #print(i.find('span', class_='section-review-publish-date').text)
        bew_datum.append(i.find('span', class_='section-review-publish-date').text)
    temp_df['Bewertung_Datum'] =  bew_datum
    exchange_column(df,temp_df,'Bewertung_Datum')

#### Collect All Sterne

In [11]:
def collect_all_sterne(sterne):
    sternebewert = []
    for i in sterne:
        sternebewert.append(i['aria-label'])
    temp_df['Sternebewertung'] =  sternebewert
    exchange_column(df,temp_df,'Sternebewertung')    

#### Collect All Bewertungen

In [12]:
def collect_all_bewertungen(bewertung):
    tex_bewertung = []
    for i in bewertung:
        tex_bewertung.append(i.text)

    temp_df['Textuelle_Bewertung'] =  tex_bewertung
    exchange_column(df,temp_df,'Textuelle_Bewertung')      

#### Name der Klinik und Klinik_ID

In [13]:
def choose_klinik(ref_value):
    return df_kliniklist.loc[df_kliniklist['Link Google Maps']==ref_value]['Klinikname'].to_string().split(" ", 1)[1].lstrip()

In [14]:
def choose_klinik_id(ref_value):
    return df_kliniklist.loc[df_kliniklist['Link Google Maps']==ref_value]['klinik_ID'].to_string().split(" ", 1)[1].lstrip()

In [15]:
def name_klinik_klinik_ID(klinik_name, klinik_id):
    name_klinik = []
    klin_id = []
    for i in range(bew_anzahl):
        name_klinik.append(klinik_name)
        klin_id.append(klinik_id)

    temp_df['Name_Klinik'] =  name_klinik
    exchange_column(df,temp_df,'Name_Klinik')      

    temp_df['Klinik_ID'] =  klin_id
    exchange_column(df,temp_df,'Klinik_ID')

#### Bewertung_ID

In [16]:
def bewertung_ID_erstell():
    df["Bewertung_ID"] = [i for i in range(1,bew_anzahl+1)]

#### Filna und NONE und Style und CSV

In [17]:
def fillna_and_none(klinik_id,df):
    df= df.replace(r'', np.NaN)
    df = df.fillna(value=np.nan)
    df = df[['Bewertung_ID', 'Klinik_ID','Name_Klinik','Bewertung_Datum','Sternebewertung','Textuelle_Bewertung','Likes']]
    path = r'csv\\' + klinik_id + '.csv'
    df.to_csv(path, encoding="utf-8", index=False)
    driver.close()

In [ ]:
for i in df_kliniklist['Link Google Maps']:
    try:
        list_of_elements = []
        column_names=['Klinik_ID','Name_Klinik','Textuelle_Bewertung','Sternebewertung','Bewertung_Datum','Likes']
        df=pd.DataFrame(columns=column_names)
        temp_df = pd.DataFrame()
        url=i
        driver=webdriver.Chrome(executable_path="../chromedriver")
        driver.get(url)
        accept_cookies()
        zahl=driver.find_element_by_css_selector("#pane > div > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show > div.jqnFjrOWMVU__root > div > div.jqnFjrOWMVU__right > div.gm2-caption").text
        bew_anzahl = int(zahl.split(" ", 1)[0])
        scroll_down(bew_anzahl)
        see_all_comments()
        list_of_elements = saving_all_comments_in_a_list()
        collect_all_likes(list_of_elements[3])
        collect_all_bewerbung_datum(list_of_elements[2])
        collect_all_sterne(list_of_elements[1])
        collect_all_bewertungen(list_of_elements[0])
        klinik_name = choose_klinik(i)
        klnk_id = choose_klinik_id(i)
        name_klinik_klinik_ID(klinik_name, klnk_id)
        bewertung_ID_erstell()
        fillna_and_none(klinik_name,df)
        print('Success')
    except:
        driver.close()
        print('End Mit Fehler(Übersprungen)')

Success
Success
Success


#### Merge All Files in One

In [ ]:
path = "csv/"
all_files = glob.glob(os.path.join(path, "*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv( "csv/merged_all.csv", encoding="utf-8", index=False)

In [ ]:
df_merged

In [2]:
import pandas as pd

In [3]:
df_all = pd.read_csv('csv/sss/merged_all.csv')

In [5]:
df_all.head(30)

,Bewertung_ID,Klinik_ID,Name_Klinik,Bewertung_Datum,Sternebewertung,Textuelle_Bewertung,Likes
0,1,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,vor 5 Tagen,5 Sterne,Wirklich tolle Menschen in der Notaufnahme. Je...,NaN
1,2,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,vor 7 Monaten,5 Sterne,"Neues, modernes Klinikum mit guter Ausstattung...",1.0
2,3,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,vor einem Monat,5 Sterne,Ich habe im Krankenhaus entbunden. Super nette...,2.0
3,4,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,vor 2 Monaten,5 Sterne,Mir hat man hier auf Grund mittelschwerer Co19...,NaN
4,5,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,vor 2 Jahren,5 Sterne,"Sollte ich mal ins Krankenhaus müssen,\ndann m...",3.0
5,6,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,vor 2 Monaten,3 Sterne,Ich habe wirklich gute Erfahrungen in der Nota...,NaN
6,7,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,vor einer Woche,5 Sterne,Meine Mutter hat bis heute Morgen auf der W12 ...,1.0
7,8,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,vor 2 Wochen,4 Sterne,Ich war Anfang des Jahres zum geplanten Kaiser...,NaN
8,9,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,vor 3 Wochen,1 Stern,Unglaublich lange Wartezeiten trotz eines Term...,1.0
9,10,16,AGAPLESION EV. KLINIKUM SCHAUMBURG,vor einem Monat,1 Stern,"toll, wenn sich erst die im Büro ungefährdete ...",NaN
